# Model calibration
This notebook contains a calibration of the model for the current number of agents. Usually ABMs are not linearly scalable and therefore this procedure should be followed for model calibration, to avoid unnecessary long start-up time of the model and thus a wastage of computing power. Because the RIVM stakeholders mentioned during one of the final meetings that they were especially interested in the process of model-building, this notebook is an annex to assist for when the model is built. Note that in case this model will be extended, this notebook will require to be extended, since it only includes current variables. However, most certainly the process stays the same.

Calibration outcomes of the model need to be implemented directly in the agents manually. Due to ABMs being stochastic, initial values will not be perfect but provide a good starting point. First we will be importing the model in the notebook and set it up in such a way that all policies and scenarios are turned off to see whether we can get the model in some sort of equilibrium.

In [1]:
from model.cepai_model import *

levers = {
    "L1": 0.0,  # Minimal requirement for reused parts
    "L2": 0.0,  # Minimal requirement for high-quality plastic
    "L3": 0.0,  # Use better solvable cohesives
    "L4": 1.0,  # Include externality for virgin plastic
    "L5": 0.05   # Minimal requirement for recyclate, calibrated for 5% according to Volvo.
        }

uncertainties = {
    "X1": 0.0,  # Annual increase factor of oil price
    "X2": 0.0,  # Annual probability for global oil shock
    "X3": 0.0   # Annual increase of recycling efficiency
        }

In [2]:
model = CEPAIModel(levers=levers, uncertainties=uncertainties)

Simulation starting...


In [3]:
model_stabilisation_run = model.run(steps=150, time_tracking=True)

Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Step: 50
Step: 51
Step: 52
Step: 53
Step: 54
Step: 55
Step: 56
Step: 57
Step: 58
Step: 59
Step: 60
Step: 61
Step: 62
Step: 63
Step: 64
Step: 65
Step: 66
Step: 67
Step: 68
Step: 69
Step: 70
Step: 71
Step: 72
Step: 73
Step: 74
Step: 75
Step: 76
Step: 77
Step: 78
Step: 79
Step: 80
Step: 81
Step: 82
Step: 83
Step: 84
Step: 85
Step: 86
Step: 87
Step: 88
Step: 89
Step: 90
Step: 91
Step: 92
Step: 93
Step: 94
Step: 95
Step: 96
Step: 97
Step: 98
Step: 99
Step: 100
Step: 101
Step: 102
Step: 103
Step: 104
Step: 105
Step: 106
Step: 107
Step: 108
Step: 109
Step: 110


And next we will run steps for gathering mean data to provide a good starting point for the experimental model runs. There are multiple things we need initial values for: stocks and demands. For garages we will not initially set up stocks with cars, as these belong to users and this will unnecessarily complicate the process.

In [4]:
def get_stocks(agent):
    agent_stocks = {}

    for component, stocks in agent.stock.items():
        if isinstance(stocks, (float or int)):
            agent_stocks[component] = stocks
        elif isinstance(stocks, list):
            agent_stocks[component] = len(stocks)

    return agent_stocks

In [5]:
import collections
import pandas as pd

calibration_steps = 50

data = []

for _ in range(calibration_steps):
    """
    Collect every step of the model the values for stocks and demands of all agent classes except users and refiners and save them in a Pandas dataframe.
    """
    # Garages, check both demand and stock
    i = 0

    for garage in model.all_agents[Garage]:
        if i == 0:
            garage_stocks  = get_stocks(garage)
            garage_demands = garage.demand
            i += 1
        else:
            garage_stock_to_add = get_stocks(garage)
            garage_stocks = dict(collections.Counter(garage_stocks) + collections.Counter(garage_stock_to_add))
            garage_demands = dict(collections.Counter(garage_demands) + collections.Counter(garage.demand))
            i += 1

        garage_stocks = {k: v / i for k, v in garage_stocks.items()}
        garage_demands = {k: v / i for k, v in garage_demands.items()}

    # Parts manufacturers, check both demand and stock
    i = 0

    for PM in model.all_agents[PartsManufacturer]:
        if i == 0:
            PM_stocks = get_stocks(PM)
            PM_demands = PM.demand
            i += 1
        else:
            PM_stock_to_add = get_stocks(PM)
            PM_stocks = dict(collections.Counter(PM_stocks) + collections.Counter(PM_stock_to_add))
            PM_demands = dict(collections.Counter(PM_demands) + collections.Counter(PM.demand))
            i += 1

        PM_stocks = {k: v / i for k, v in PM_stocks.items()}
        PM_demands = {k: v / i for k, v in PM_demands.items()}

    # Car manufacturers, check both demand and stock
    i = 0

    for CM in model.all_agents[CarManufacturer]:
        if i == 0:
            CM_stocks = get_stocks(CM)
            CM_demands = CM.demand
            i += 1
        else:
            CM_stock_to_add = get_stocks(CM)
            CM_stocks = dict(collections.Counter(CM_stocks) + collections.Counter(CM_stock_to_add))
            CM_demands = dict(collections.Counter(CM_demands) + collections.Counter(CM.demand))
            i += 1

        CM_stocks = {k: v / i for k, v in CM_stocks.items()}
        CM_demands = {k: v / i for k, v in CM_demands.items()}

    # Dismantlers, check stock
    i = 0

    for dismantler in model.all_agents[Dismantler]:
        if i == 0:
            dismantler_stocks = get_stocks(dismantler)
            i += 1
        else:
            dismantler_stock_to_add = get_stocks(dismantler)
            dismantler_stocks = dict(collections.Counter(dismantler_stocks) + collections.Counter(dismantler_stock_to_add))
            i += 1

        garage_stocks = {k: v / i for k, v in dismantler_stocks.items()}

    # Recyclers, check stock
    i = 0

    for recycler in model.all_agents[Recycler]:
        if i == 0:
            recycler_stocks = get_stocks(recycler)
            i += 1
        else:
            recycler_stock_to_add = get_stocks(recycler)
            recycler_stocks = dict(collections.Counter(recycler_stocks) + collections.Counter(recycler_stock_to_add))
            i += 1

        recycler_stocks = {k: v / i for k, v in recycler_stocks.items()}

    # Update the dataframes
    update_list = [garage_stocks, garage_demands, PM_stocks, PM_demands, CM_stocks, CM_demands, dismantler_stocks, recycler_stocks]
    data.append(update_list)

    # And advance the model one step again.
    model.step()

columns = ['garage_stock', 'garage_demand', 'PM_stock', 'PM_demand', 'CM_stock', 'CM_demand', 'dismantler_stock', 'recycler_stock']
calibration_df = pd.DataFrame(data=data, columns=columns)

See below a brief overview of the TOTAL (not individual) demands and stocks of the last 50 steps.

In [6]:
n_steps = len(calibration_df)

# For every column
for column in calibration_df:
    # Retrieve means of the calibration steps and save in another dataframe.
    mean_steps = {}

    for step in calibration_df[column]:
        mean_steps = dict(collections.Counter(mean_steps) + collections.Counter(step))

    mean_steps = {k: v / n_steps for k, v in mean_steps.items()}
    print(column, mean_steps)

garage_stock {<Component.PARTS_FOR_RECYCLER: 5>: 5.1, <Component.PARTS: 4>: 38.28}
garage_demand {<Component.PARTS: 4>: 105.96}
PM_stock {<Component.VIRGIN: 1>: 2.678351955335929, <Component.RECYCLATE_HIGH: 3>: 0.025781851406327636, <Component.PARTS: 4>: 9622.333333333332, <Component.RECYCLATE_LOW: 2>: 3.227973444097643e-16}
PM_demand {<Component.VIRGIN: 1>: 101.75821314134316, <Component.RECYCLATE_LOW: 2>: 6.058453525323516, <Component.PARTS: 4>: 107.81666666666666}
CM_stock {<Component.CARS: 6>: 11.670833333333333}
CM_demand {<Component.PARTS: 4>: 31.00333333333334, <Component.CARS: 6>: 7.750833333333335}
dismantler_stock {<Component.PARTS_FOR_RECYCLER: 5>: 5.1, <Component.PARTS: 4>: 38.28}
recycler_stock {<Component.RECYCLATE_LOW: 2>: 9.43422832682544, <Component.RECYCLATE_HIGH: 3>: 52996.18944055571}


In [90]:
car_count = 0
to_buy_car_count = 0

for user in model.all_agents[User]:
    if user.stock[Component.CARS]:
        #print(user.stock[Component.CARS])
        car_count += 1
    elif user.demand[Component.CARS] == 1:
        to_buy_car_count += 1

In [91]:
print(car_count, to_buy_car_count)

485 17


In [92]:
model.step()

In [93]:
for PM in model.all_agents[PartsManufacturer]:
    print("parts demand", PM.demand[Component.PARTS])
    print("parts stock", len(PM.stock[Component.PARTS]))
    print(PM.stock[Component.VIRGIN])
    print(PM.stock[Component.RECYCLATE_HIGH])
    print(PM.stock[Component.RECYCLATE_LOW])
    print(PM.sold_volume)
    print("")

parts demand 114
parts stock 107
11.88160813774243
4.440892098500626e-16
0.056541662562187706
{'last': 853, 'second_last': 770}

parts demand 94
parts stock 82
2.0074348583347885
4.440892098500626e-16
0.05382097117687445
{'last': 927, 'second_last': 847}

parts demand 107
parts stock 103
20.971431643943234
0.0
7.0360384185619296e-15
{'last': 746, 'second_last': 653}



In [71]:
for CM in model.all_agents[CarManufacturer]:
    print(CM.demand[Component.CARS])
    print(len(CM.stock[Component.PARTS]))
    print(len(CM.stock[Component.CARS]))
    print(CM.sold_volume)
    print("")

48
2
53
{'last': 36, 'second_last': 44}

13
0
19
{'last': 0, 'second_last': 0}

9
0
18
{'last': 0, 'second_last': 0}

17
2
21
{'last': 0, 'second_last': 17}



In [48]:
for garage in model.all_agents[Garage]:
    print(garage.stock[Component.CARS])
    print(len(garage.stock[Component.PARTS]))

[<model.bigger_components.Car object at 0x0000019FD22EB190>, <model.bigger_components.Car object at 0x0000019FD21F4B20>, <model.bigger_components.Car object at 0x0000019FD1CC9F00>, <model.bigger_components.Car object at 0x0000019FD2012A70>, <model.bigger_components.Car object at 0x0000019FD1CC9ED0>, <model.bigger_components.Car object at 0x0000019FD1F51C00>, <model.bigger_components.Car object at 0x0000019FD22B4FA0>, <model.bigger_components.Car object at 0x0000019FD22B4DF0>, <model.bigger_components.Car object at 0x0000019FD20A6DD0>, <model.bigger_components.Car object at 0x0000019FD22B66E0>, <model.bigger_components.Car object at 0x0000019FD2466800>, <model.bigger_components.Car object at 0x0000019FD21C6BC0>, <model.bigger_components.Car object at 0x0000019FD22EB4C0>, <model.bigger_components.Car object at 0x0000019FD2462B30>, <model.bigger_components.Car object at 0x0000019FD2012890>, <model.bigger_components.Car object at 0x0000019FD1CCACB0>, <model.bigger_components.Car object at 